In [138]:
def HMM_Alignment(seq_1, seq_2, deltas, rhos, epsilons, tau, match=0.9, missmatch=0.1, gap=1):
    
    import numpy as np
    import pandas as pd
    from itertools import product
    
    def cost(ch_1, ch_2):
        return 'M' if ch_1==ch_2 else 'Mm'
    
    n=len(seq_1)
    m=len(seq_2)
    
    Transit=pd.DataFrame([[np.nan, 1-deltas[0]-deltas[1], deltas[0], deltas[1], np.nan],
                            [np.nan, 1-deltas[0]-deltas[1]-tau, deltas[0], deltas[1], tau], 
                            [np.nan, 1-epsilons[0]-rhos[1]-tau, epsilons[0], rhos[1], tau], 
                            [np.nan, 1-epsilons[1]-rhos[0]-tau, rhos[0], epsilons[1], tau], 
                            [np.nan, np.nan, np.nan, np.nan, 1]],
                            index=['B', 'M', 'X', 'Y', 'E'], columns=['B', 'M', 'X', 'Y', 'E'])
    
    Emission=pd.Series([match, missmatch, gap, gap], index=['M', 'Mm', 'X', 'Y'])
    
    M_mat=np.full([n+1, m+1], np.nan)
    X_mat=np.full([n+1, m+1], np.nan)
    Y_mat=np.full([n+1, m+1], np.nan)
    
    X_mat[0, 1:]=[Transit.loc['B', 'X']*Transit.loc['X', 'X']**j for j in range(m)]
    X_mat[1:, 0]=[0]*n
    Y_mat[1:, 0]=[Transit.loc['B', 'Y']*Transit.loc['Y', 'Y']**i for i in range(n)]
    Y_mat[0, 1:]=[0]*m 
    M_mat[1:, 0]=[0]*n
    M_mat[0, 1:]=[0]*m
    M_mat[1, 1]=Transit.loc['B', 'M']*Emission[cost(seq_1[0], seq_2[0])]

    M_matb=np.full([n+1, m+1], np.nan)
    X_matb=np.full([n+1, m+1], np.nan)
    Y_matb=np.full([n+1, m+1], np.nan)

    X_matb[n, :]=[Transit.loc['X', 'X']**j for j in range(m, -1, -1)]
    Y_matb[n, :]=[X_matb[n, j+1]*Transit.loc['Y', 'X'] if j+1!=m+1 else 1 for j in range(m, -1, -1)][::-1]
    M_matb[n, :]=[X_matb[n, j+1]*Transit.loc['M', 'X'] if j+1!=m+1 else 1 for j in range(m, -1, -1)][::-1]
    
    Y_matb[:, m]=[Transit.loc['Y', 'Y']**i for i in range(n, -1, -1)]
    X_matb[:, m]=[Y_matb[i+1, m]*Transit.loc['X', 'Y'] if i+1!=n+1 else 1 for i in range(n, -1, -1)][::-1]
    M_matb[:, m]=[Y_matb[i+1, m]*Transit.loc['M', 'Y'] if i+1!=n+1 else 1 for i in range(n, -1, -1)][::-1]

    def forward_pass(M_mat, X_mat, Y_mat):
        for i, j in product(range(1, n+1), range(1, m+1)):
            ch_1, ch_2=seq_1[i-1], seq_2[j-1]
            if (i, j)!=(1, 1):
                M_mat[i, j]=M_mat[i-1, j-1]*Transit.loc['M', 'M']*Emission[cost(ch_1, ch_2)]+\
                            X_mat[i-1, j-1]*Transit.loc['X', 'M']*Emission[cost(ch_1, ch_2)]+\
                            Y_mat[i-1, j-1]*Transit.loc['Y', 'M']*Emission[cost(ch_1, ch_2)]
                    
            X_mat[i, j]=M_mat[i, j-1]*Transit.loc['M', 'X']*Emission['X']+\
                        X_mat[i, j-1]*Transit.loc['X', 'X']*Emission['X']+\
                        Y_mat[i, j-1]*Transit.loc['Y', 'X']*Emission['X']
                    
            Y_mat[i, j]=M_mat[i-1, j]*Transit.loc['M', 'Y']*Emission['Y']+\
                        X_mat[i-1, j]*Transit.loc['X', 'Y']*Emission['Y']+\
                        Y_mat[i-1, j]*Transit.loc['Y', 'Y']*Emission['Y']
        
        return M_mat, X_mat, Y_mat 
            
    def backward_pass(M_matb, X_matb, Y_matb):
        for i, j in product(range(n-1, -1, -1), range(m-1, -1, -1)):
            ch_1, ch_2=seq_1[i], seq_2[j]
            
            M_matb[i, j]=M_mat[i+1, j+1]*Transit.loc['M', 'M']*Emission[cost(ch_1, ch_2)]+\
                        X_mat[i, j+1]*Transit.loc['M', 'X']*Emission['X']+\
                        Y_mat[i+1, j]*Transit.loc['M', 'Y']*Emission['Y']
                    
            X_matb[i, j]=M_mat[i+1, j+1]*Transit.loc['X', 'M']*Emission[cost(ch_1, ch_2)]+\
                        X_mat[i, j+1]*Transit.loc['X', 'X']*Emission['X']+\
                        Y_mat[i+1, j]*Transit.loc['X', 'Y']*Emission['Y']
                    
            Y_matb[i, j]=M_mat[i+1, j+1]*Transit.loc['Y', 'M']*Emission[cost(ch_1, ch_2)]+\
                        X_mat[i, j+1]*Transit.loc['Y', 'X']*Emission['X']+\
                        Y_mat[i+1, j]*Transit.loc['Y', 'Y']*Emission['Y']

        return M_matb, X_matb, Y_matb
    
    def FB_probs(M_mat, X_mat, Y_mat, M_matb, X_matb, Y_matb):
        Denum=np.multiply(M_mat, M_matb)+np.multiply(X_mat, X_matb)+np.multiply(Y_mat, Y_matb)
        M_probs=np.multiply(M_mat, M_matb)/Denum
        X_probs=np.multiply(X_mat, X_matb)/Denum
        Y_probs=np.multiply(Y_mat, Y_matb)/Denum
        return np.round(M_probs, 2), np.round(X_probs, 2), np.round(Y_probs, 2)
    
    M_mat, X_mat, Y_mat=forward_pass(M_mat, X_mat, Y_mat)
    M_matb, X_matb, Y_matb=backward_pass(M_matb, X_matb, Y_matb)
    M_probs, X_probs, Y_probs=FB_probs(M_mat, X_mat, Y_mat, M_matb, X_matb, Y_matb)
    
    return Transit, M_probs, X_probs, Y_probs, M_mat, X_mat, Y_mat, M_matb, X_matb, Y_matb

In [146]:
T, Mp, Xp, Yp, M, X, Y, Mb, Xb, Yb=HMM_Alignment('AGAGA', 'AGAGAGA', deltas=[0.1]*2, rhos=[0.1]*2, epsilons=[0.3]*2,
             tau=0.1)

In [147]:
print('Match matrix:\n', Mp)
print('Horizontal gap matrix:\n', Xp)
print('Vertical gap matrix:\n', Yp)

Match matrix:
 [[  nan  0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.98  0.03  0.39  0.02  0.34  0.02  0.26]
 [ 0.    0.03  0.97  0.03  0.58  0.03  0.51  0.05]
 [ 0.    0.39  0.03  0.97  0.04  0.68  0.03  0.54]
 [ 0.    0.02  0.58  0.04  0.96  0.04  0.74  0.06]
 [ 0.    0.3   0.06  0.6   0.06  0.91  0.03  0.74]]
Horizontal gap matrix:
 [[  nan  1.    1.    1.    1.    1.    1.    1.  ]
 [ 0.    0.01  0.95  0.59  0.96  0.64  0.97  0.68]
 [ 0.    0.02  0.01  0.92  0.39  0.93  0.47  0.83]
 [ 0.    0.02  0.05  0.02  0.89  0.3   0.92  0.39]
 [ 0.    0.02  0.02  0.07  0.02  0.87  0.24  0.71]
 [ 0.    0.06  0.14  0.08  0.3   0.07  0.91  0.23]]
Vertical gap matrix:
 [[  nan  0.    0.    0.    0.    0.    0.    0.  ]
 [ 1.    0.01  0.02  0.02  0.02  0.02  0.02  0.05]
 [ 1.    0.95  0.01  0.05  0.02  0.04  0.02  0.12]
 [ 1.    0.59  0.92  0.02  0.07  0.02  0.05  0.07]
 [ 1.    0.96  0.39  0.89  0.02  0.09  0.02  0.23]
 [ 1.    0.64  0.8   0.32  0.64  0.02  0.06  0.03]]


In [148]:
T, Mp, Xp, Yp, M, X, Y, Mb, Xb, Yb=HMM_Alignment('ATAGCTACGAC', 'TGCTAGCTAGC', deltas=[0.1]*2, rhos=[0.1]*2, epsilons=[0.3]*2,
             tau=0.1)

In [149]:
print('Match matrix:\n', Mp)
print('Horizontal gap matrix:\n', Xp)
print('Vertical gap matrix:\n', Yp)

Match matrix:
 [[  nan  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.69  0.14  0.25  0.11  0.5   0.06  0.12  0.05  0.32  0.03  0.07]
 [ 0.    0.62  0.27  0.13  0.75  0.06  0.17  0.08  0.66  0.05  0.1   0.1 ]
 [ 0.    0.15  0.39  0.23  0.13  0.91  0.06  0.07  0.06  0.8   0.04  0.09]
 [ 0.    0.06  0.81  0.17  0.13  0.07  0.95  0.04  0.05  0.05  0.84  0.07]
 [ 0.    0.06  0.08  0.93  0.07  0.07  0.05  0.96  0.04  0.04  0.04  0.79]
 [ 0.    0.36  0.07  0.05  0.95  0.05  0.05  0.04  0.96  0.04  0.04  0.05]
 [ 0.    0.04  0.29  0.06  0.04  0.92  0.09  0.04  0.04  0.91  0.09  0.07]
 [ 0.    0.09  0.08  0.41  0.04  0.09  0.55  0.58  0.06  0.09  0.55  0.67]
 [ 0.    0.04  0.44  0.04  0.22  0.04  0.64  0.15  0.38  0.06  0.64  0.15]
 [ 0.    0.03  0.11  0.11  0.06  0.33  0.16  0.28  0.14  0.5   0.16  0.26]
 [ 0.    0.07  0.09  0.5   0.1   0.07  0.21  0.63  0.13  0.08  0.25  0.73]]
Horizontal gap matrix:
 [[  nan  1.    1.    1.    1.    1.    1.    1.    1.    1. 